# Чтение и запись данных. Часть 1

## 1. Чтение текстовых файлов, файловые дескрипторы

### 1.1

**Простая задача** сформируйте объект класса dict, где ключ - это IP адрес, а значение - количество раз, которые этот ip встретился в тесте.

*Результат выполнения*
<pre>
{'192.168.101.4': 1, '192.168.102.3': 2, '192.168.7.46': 1}
</pre>

In [2]:
file_path = './data/uwsgi.log'
ip_counter = dict()

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        parted_row = row.split(' ')
        if len(parted_row) > 1:
            ip_counter[parted_row[0]] = ip_counter.setdefault(parted_row[0], 0) + 1
            
print(ip_counter)

{'192.168.101.4': 1, '192.168.102.3': 2, '192.168.7.46': 1}


### 1.2
**Задача среднего уровня** Распечатайте список URL, к которым происходил доступ. GET или POST параметры печатать не нужно.

*Результат выполнения*
<pre>
/movie/is_personalizable/
/logger/content/time/
/movie/collection/items/recommendations/
/movie/recommendations/
</pre>

In [27]:
file_path = './data/uwsgi.log'

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        row = row.replace(' ', '?')
        res_row = None
        
        # Берём при поиске символы перед и после get, чтобы обойти ситуацию, 
        # если в параметрах post запроса есть слово get.
        # здесь бы воспользоваться регэкспами, но не добралась до них ещё.
        if '?"GET?' in row:
            res_row = row[row.find('GET') + 4:]
            res_row = res_row[:res_row.find('?')]
        elif '?"POST?' in row:
            res_row = row[row.find('POST') + 5:]
            res_row = res_row[:res_row.find('?')]
           
        print(row)
        if res_row:
            print(res_row)

192.168.101.4?-?-?[05/Feb/2019:21:36:07?+0300]?"GET?/movie/is_personalizable/?history_type=watch&uid=5734473158358418&master_uid=5734473158358418?HTTP/1.1"?200?75?"-"?"python-requests/2.0.0?CPython/2.7.3?Linux/4.4.0-47-generic"

/movie/is_personalizable/
192.168.102.3?-?-?[05/Feb/2019:21:36:07?+0300]?"POST?/logger/content/time/?HTTP/1.1"?404?305?"-"?"Mozilla/5.0?(Windows;?U;?en-US;?rv:1.8.1.11;?Gecko/20071129;?Firefox/2.5.0)?Maple?6.0.00067?Navi"

/logger/content/time/
192.168.102.3?-?-?[05/Feb/2019:21:36:07?+0300]?"GET?/movie/collection/items/recommendations/?uid=1623029046&master_uid=1623029046&collection_id=1525&subsite=141&app_version=10924&user_ab_bucket=10679?HTTP/1.1"?200?535?"-"?"python-requests/2.0.0?CPython/2.7.3?Linux/3.13.0-24-generic"

/movie/collection/items/recommendations/
192.168.7.46?-?-?[05/Feb/2019:21:36:07?+0300]?"GET?/movie/recommendations/?uid=803285924&master_uid=803285924&iid=102751&user_ab_bucket=12493&top=30&scenario_id=ITEM_PAGE&app_version=15100&subsite=302

### 1.3
**Задача высокого уровня** Сформируйте объект класса dict, где ключ - это URL, а значение - это массив объектов python `datetime.datetime`: когда происходили запросы к этому URL.

*Результат выполнения*
<pre>
{
    '/movie/is_personalizable/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/logger/content/time/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/movie/collection/items/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)],
    '/movie/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)]
}
</pre>

In [32]:
import datetime
file_path = './data/uwsgi.log'

access_journal = dict()

with open(file_path, 'r', encoding='utf8') as f:
    for row in f:
        row = row.replace(' ', '?')
        res_url = None
        
        if '?"GET?' in row:
            res_url = row[row.find('GET') + 4:]
            res_url = res_url[:res_url.find('?')]
        elif '?"POST?' in row:
            res_url = row[row.find('POST') + 5:]
            res_url = res_url[:res_url.find('?')]
           
        if res_url:
            # [05/Feb/2019:21:36:07?+0300]
            cur_val = access_journal.setdefault(res_url, [])
            res_date_str = row.split('[')
            if len(res_date_str) > 1:
                res_date_str = res_date_str[1][:res_date_str[1].find('?')]
                res_date_obj = datetime.datetime.strptime(res_date_str, '%d/%b/%Y:%H:%M:%S')
            else:
                res_date_obj = None
            cur_val.append(res_date_obj)
            
print(access_journal)

{'/movie/is_personalizable/': [datetime.datetime(2019, 2, 5, 21, 36, 7)], '/logger/content/time/': [datetime.datetime(2019, 2, 5, 21, 36, 7)], '/movie/collection/items/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)], '/movie/recommendations/': [datetime.datetime(2019, 2, 5, 21, 36, 7)]}


## 2. Чтение csv файлов: модуль python csv

### 2.1
**Простая задача** Сформируйте словарь, в котором ключ - название столбца, а значение - массив записей в этом столбце.

*Результат выполнения*
<pre>
{
    'Код': ['HYDRA-535', 'HYDRA-534', 'HYDRA-532', 'HYDRA-531', 'HYDRA-530', 'HYDRA-527', 'HYDRA-524', 'HYDRA-523', 'HYDRA-520', 'HYDRA-519', 'HYDRA-518', 'HYDRA-517', 'HYDRA-514', 'HYDRA-513', 'HYDRA-511', 'HYDRA-510', 'HYDRA-509', 'HYDRA-507', 'HYDRA-506'],
    'Тема': ['Пробрасывать пользовательское распределение paid_types в ехидну', 'Гибридный рекомендатель с multi-channel feedback', 'Джоба в дженкинсе для расчёта динамики РВП', 'Интеграция Hydra с Gamora', 'Тестируем интеграцию с Jira', 'Поправить функцию _get_ui_rec_matrix', 'Оптимизировать матрицу ItemFactors', 'Сортировка ЦПБ', 'Закостылить параметр top', "Сделать 'stable' конфигом по умолчанию в Гидре", 'Неудобно тестировать запись в redis', 'Улучшить рекомендации (первая итерация)', 'Добавить логирование в скрипты hydra/bin', 'Поправить storage_backend', 'Перемешивание рекомендаций для старых пользователей', 'Поправить скрипты bpr и и оценщика', 'Динамические персональные рекомендации', 'Навести порядок в prepare_data_for_hydra', 'Техдолг по логике /collection/recommendations/'],
    'Компонент': ['echidna', 'hydra', 'hydramatrices', 'hydramagrices', 'hydra', 'hydra', 'hydra', 'hydra', 'hydra', 'hydra', 'hydramatrices', 'hydra', 'hydramagrices', 'hydramatrices', 'hydra', 'hydra_utils', 'hydra', 'hydramagrices', 'hydra'],
    'Затрачено в часах': ['1', '3', '2', '4', '2', '10', '2', '5', '2', '2', '1', '7', '5', '2', '5', '16', '10', '3', '24']
}
</pre>

In [138]:
import csv
columns_dict = dict()
file_path = './data/task.csv'


with open(file_path, newline='', encoding='utf8') as csvfile:
    reader = csv.DictReader(csvfile)
    for col_name in reader.fieldnames:
        columns_dict[col_name] = []
    for row in reader:
        for key, value in row.items():
            columns_dict[key].append(value)

print(columns_dict)

{'Код': ['HYDRA-535', 'HYDRA-534', 'HYDRA-532', 'HYDRA-531', 'HYDRA-530', 'HYDRA-527', 'HYDRA-524', 'HYDRA-523', 'HYDRA-520', 'HYDRA-519', 'HYDRA-518', 'HYDRA-517', 'HYDRA-514', 'HYDRA-513', 'HYDRA-511', 'HYDRA-510', 'HYDRA-509', 'HYDRA-507', 'HYDRA-506'], 'Тема': ['Пробрасывать пользовательское распределение paid_types в ехидну', 'Гибридный рекомендатель с multi-channel feedback', 'Джоба в дженкинсе для расчёта динамики РВП', 'Интеграция Hydra с Gamora', 'Тестируем интеграцию с Jira', 'Поправить функцию _get_ui_rec_matrix', 'Оптимизировать матрицу ItemFactors', 'Сортировка ЦПБ', 'Закостылить параметр top', "Сделать 'stable' конфигом по умолчанию в Гидре", 'Неудобно тестировать запись в redis', 'Улучшить рекомендации (первая итерация)', 'Добавить логирование в скрипты hydra/bin', 'Поправить storage_backend', 'Перемешивание рекомендаций для старых пользователей', 'Поправить скрипты bpr и и оценщика', 'Динамические персональные рекомендации', 'Навести порядок в prepare_data_for_hydra', '

### 2.2
**Задача среднего уровня** Посчитайте статистику по колонке "Затрачено в часах" (это человеко-часы, затраченные на задачу) – максимальное значение, минимальное значение, среднее значение. Воспользуйтесь словарём `columns_dict` из задания 2.1

*Результат выполнения*
<pre>
min_val=1, max_val=24, mean_val=5.578947368421052

</pre>

In [58]:
import pandas as pd

min_val, max_val, mean_val = None, None, None


df = pd.DataFrame(columns_dict)
df['Затрачено в часах'] = pd.to_numeric(df['Затрачено в часах'])

min_val = df['Затрачено в часах'].min()
max_val = df['Затрачено в часах'].max(axis = 0)
mean_val = df['Затрачено в часах'].mean()

print("min_val={}, max_val={}, mean_val={}".format(min_val, max_val, mean_val))


min_val=1, max_val=24, mean_val=5.578947368421052


### 2.3
**Задача высокого уровня** Сколько человеко-часов было потрачено на задачи, в которых встречается слово "рекомендации" в любых формах, а сколько на остальные задачи (в процентах)?

Подсказка - проверка на наличие нужного слова в тексте `txt` выглядит так:

In [60]:
print('рек' in 'Я делаю рекомендательную систему', 'рек' in 'кек')

True False


*Результат выполнения*
<pre>
{'recs': 2, 'non_recs': 17}
</pre>

In [137]:
file_path = './data/task.csv'
task_recs_counter = {'recs': 0, 'non_recs': 0}

with open(file_path, newline='', encoding='utf8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for key, value in row.items():
            if key == 'Тема':
                if 'рекомендаци' in value:
                    task_recs_counter['recs'] += 1
                else:
                    task_recs_counter['non_recs'] += 1

print(task_recs_counter)

# проверила, их действительно 3, что не совпадает с ожидаемым результатом

{'recs': 3, 'non_recs': 16}


## 3. Чтение csv файлов: модуль pandas

### 3.1

**Простое задание** добавьте столбец `num_tasks`, где отражено количество задач, выполненное внутри каждого компонента. Колонка `mean_val` со средним значением поля 'Затрачено в часах' уже добавлена в качестве примера.

Чтобы выполнить задание, поправьте словарь `aggregation_json`

In [88]:
import numpy as np
import pandas as pd

file_path = './data/task.csv'

df = pd.read_csv(file_path, encoding='utf8')

df.groupby('Компонент').agg(num_tasks=('Затрачено в часах', 'count'))

# код с примером закомментировала, так как в последних версиях, как я поняла, он не работает, синтаксис поменялся.
# aggregation_json = {
#  'Затрачено в часах': {
#   'mean_val': np.mean}
# }

# df.groupby(by=['Компонент'])[['Затрачено в часах']].aggregate(aggregation_json)

,num_tasks
Компонент,
echidna,1
hydra,11
hydra_utils,1
hydramagrices,3
hydramatrices,3


### 3.2
**Задание среднего уровня** Добавьте (в виде `list`) для каждого компонента набор задач, который был в этом компоненте выполнен (идентификаторы содержатся в поле "Код"). Для этого замените `.aggregate()` на `.apply(list)`

Пример:

In [93]:
import pandas as pd

print('Исходный DataFrame:')
a = pd.DataFrame({'key':['a', 'a'], 'val':[1, 2]})
print(a)
print('Применяем apply:')
a.groupby('key')['val'].apply(list)

Исходный DataFrame:
  key  val
0   a    1
1   a    2
Применяем apply:


key
a    [1, 2]
Name: val, dtype: object

Результат выполнения

<pre>
Компонент
echidna                                                [HYDRA-535]
hydra            [HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H...
hydra_utils                                            [HYDRA-510]
hydramagrices                    [HYDRA-531, HYDRA-514, HYDRA-507]
hydramatrices                    [HYDRA-532, HYDRA-518, HYDRA-513]
</pre>

In [95]:
codes_list_df = df.groupby('Компонент')['Код'].apply(list)

codes_list_df.head()

Компонент
echidna                                                [HYDRA-535]
hydra            [HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H...
hydra_utils                                            [HYDRA-510]
hydramagrices                    [HYDRA-531, HYDRA-514, HYDRA-507]
hydramatrices                    [HYDRA-532, HYDRA-518, HYDRA-513]
Name: Код, dtype: object

### 3.3
**Задание высокого уровня** Составьте DataFrame с двумя колонками: среднее время на задачу и список задач `codes_list_df` внутри каждого "Компонента". Подсказка: подготовьте два датафрейма и используйте [функцию merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

In [99]:
file_path = './data/task.csv'

df = pd.read_csv(file_path, encoding='utf8')

num_tasks_df = df.groupby('Компонент').agg(num_tasks=('Затрачено в часах', 'count'))
codes_list_df = pd.DataFrame(df.groupby('Компонент')['Код'].apply(list))

num_tasks_df.merge(codes_list_df, left_on='Компонент', right_on='Компонент')

,num_tasks,Код
Компонент,,
echidna,1,[HYDRA-535]
hydra,11,"[HYDRA-534, HYDRA-530, HYDRA-527, HYDRA-524, H..."
hydra_utils,1,[HYDRA-510]
hydramagrices,3,"[HYDRA-531, HYDRA-514, HYDRA-507]"
hydramatrices,3,"[HYDRA-532, HYDRA-518, HYDRA-513]"


## 4.  Работа с данными формата HTML

### 4.1
**Задание простого уровня** Распечатайте список курсов по программированию от Skillbox.

In [136]:
import requests
from lxml import html

page = requests.get('https://skillbox.ru/courses/code').content
pageStr = page.decode("utf-8")
#print(pageStr)
#parser = html.HTMLParser(encoding='utf-8')
html_tree = html.fromstring(pageStr)

# профессий или курсов? Вывела курсов. Закомментировано - профессий.
# items_list = html_tree.xpath("//b[contains(@class, '{}')]".format('profession-card__title hover-card__text'))
items_list = html_tree.xpath("//b[contains(@class, '{}')]".format('line-card__title hover-card__text'))
for item in items_list:
    print(item.text)


      Веб-разработчик с нуля до PRO
    

      Java-разработчик
    

      Python-разработчик
    

      Аналитик данных на Python
    

      Frontend-разработчик
    

      PHP-разработчик с нуля до PRO
    

      Мобильный разработчик PRO
    

      Фреймворк Vue.js
    

      React.js
    

      Веб-вёрстка
    

      Python-фреймворк Django
    

      Тестирование мобильных приложений
    

      Специалист по кибербезопасности
    

      Основы математики для Data Science
    

      Как подготовиться к профессиональной конференции
    

      PHP-фреймворк Symfony
    

      Middle-разработчик игр на Unity
    

      Работа в командной строке Bash
    

      SQL для анализа данных
    

      Node.js
    

      Power BI
    


### 4.2

**Задание среднего уровня**. Выведите список всех курсов Skillbox по программированию, дизайну, маркетингу и управлению. Добавьте к каждому курсу ссылку http на этот курс на сайте Skillbox.

*Результат выполнения*
<pre>
Курс: Веб-дизайн с нуля до PRO  link:  https://skillbox.ru/webdesign/
Курс: Рекламная графика  link:  https://skillbox.ru/cpeople/
Курс: UX-дизайн  link:  https://skillbox.ru/aic/
</pre>

In [139]:
import requests
from lxml import html

page = requests.get('https://skillbox.ru').content.decode('UTF-8')
html_tree = html.fromstring(page)
items = html_tree.xpath("//a[contains(@class, '{}')]".format('tab tab--link tab--main'))

print('Список курсов по разделам:\n')

for item in items:
    cur_link = item.attrib['href']
    print('\n' + item.text.replace('\n', '').strip() + ': \n')

    page = requests.get('https://skillbox.ru' + cur_link).content
    pageStr = page.decode("utf-8")

    # parser = html.HTMLParser(encoding='utf-8')
    html_tree = html.fromstring(pageStr)

    courses_list = html_tree.xpath("//div[@class='{}'] ".format('courses-list'))

    for course in courses_list:
        course_tags = course.xpath("//a[@class='{}']".format('hover-card line-card line-card--add'))
        for course_tag in course_tags:
            title_xpath = course_tag.xpath("div/b[@class='{}']".format('line-card__title hover-card__text'))
            # title = title_xpath[0].text[6:-5]
            title = title_xpath[0].text.replace('\n', '').strip()
            link = course_tag.attrib['href']
            print(f'Курс: {title}, link: {link}')

Список курсов по разделам:


Программирование: 

Курс: Веб-разработчик с нуля до PRO, link: https://skillbox.ru/course/webdev/
Курс: Java-разработчик, link: https://skillbox.ru/course/java/
Курс: Python-разработчик, link: https://skillbox.ru/course/python/
Курс: Аналитик данных на Python, link: https://skillbox.ru/python-data/
Курс: Frontend-разработчик, link: https://skillbox.ru/course/frontend-developer/
Курс: PHP-разработчик с нуля до PRO, link: https://skillbox.ru/course/php/
Курс: Мобильный разработчик PRO, link: https://skillbox.ru/agima/
Курс: Фреймворк Vue.js, link: https://skillbox.ru/course/vue-js/
Курс: React.js, link: https://skillbox.ru/course/react-js/
Курс: Веб-вёрстка, link: https://skillbox.ru/course/weblayout/
Курс: Python-фреймворк Django, link: https://skillbox.ru/course/django-framework/
Курс: Тестирование мобильных приложений, link: https://skillbox.ru/course/mobile-testing/
Курс: Специалист по кибербезопасности, link: https://skillbox.ru/course/cybersecurity/
Кур

### 4.3
**Задание высокого уровня**. Напишите код, который переходит по ссылкам на страницы курсов по программированию из предыдущего задания и распечатывает описание курса.

In [140]:
import requests
from lxml import html
import pandas as pd

links = []
descr = []

page = requests.get('https://skillbox.ru/code/').content.decode('utf-8')
html_tree = html.fromstring(page)

courses_list = html_tree.xpath("//div[@class='{}'] ".format('courses-list'))

for course in courses_list:
    course_tags = course.xpath("//a[@class='{}']".format('hover-card line-card line-card--add'))
    for course_tag in course_tags:
        cur_link = course_tag.attrib['href']
        links.append(cur_link)

        course_page = requests.get(cur_link).content.decode('utf-8')
        course_html_tree = html.fromstring(course_page)
        cur_descr = course_html_tree.xpath("//p[@class='{}'] ".format('start-screen__desc'))[0].text.strip()
        descr.append(cur_descr)

result_df = pd.DataFrame({
    'link': links,
    'descr': descr
})

result_df.head()

,link,descr
0,https://skillbox.ru/course/webdev/,"Вы научитесь верстать сайты на HTML и CSS, изу..."
1,https://skillbox.ru/course/java/,Вы научитесь писать код и создавать сайты на с...
2,https://skillbox.ru/course/python/,Вы научитесь писать чистый код и создавать сло...
3,https://skillbox.ru/python-data/,Освойте Python и делайте свою работу в несколь...
4,https://skillbox.ru/course/frontend-developer/,"Вы изучите основы HTML, CSS и JavaScript, науч..."


## 5. Работа с данными формата XML

### 5.1

**Задание начального уровня** Распечатайте все фильмы (а не только драмы) в формате: `название фильма: жанр`

In [143]:
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'

with open(file_path) as f:
    doc = ElementTree.parse(f)
    content_titles = doc.getroot()
    for movie in content_titles.findall('./Content/content_title'):
        genre = 'unknown'
        if movie.find('./genre') is not None:
            genre = movie.find('./genre').text
        print(movie.find('./title').text + ': ' + genre)

The Shawshank Redemption: drama
The Dark Knight: drama
"Back 2 the Future": unknown


### 5.2
**Задание среднего уровня** Посчитайте среднюю длительность контента в файле. xml_content_description

In [147]:
import math
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'
duration = []

num_content = 0

with open(file_path) as f:
    doc = ElementTree.parse(f)
    content_titles = doc.getroot()
    for movie in content_titles.findall('./Content/content_title'):
        if movie.find('./duration') is not None:
            num_content += 1 # Контентом для вычислений считаем только те фильмы, у который duration указан
            duration.append(int(movie.find('./duration').text))

if num_content:
    print("avg_duration={}".format(sum(duration)/num_content))

avg_duration=8820.0


### 5.3
**Задание высокого уровня** Если речь идёт о продакшн-системах машинного обучения, то данные могут приходить в форматах, мало пригодных для использования - например XML. В таких случаях приходится приводить данные к превычному табличному формату - такой кейс мы решим в рамках домашней работы.

Нужно определить, сколько полей будет в выходном файле и как залить в него данные

Алгоритм
* сформируйте массив словарей `content_list`, где каждый элемент массива - это словарь вида `{tag1: value1,...,tag_n:value_n}`
* каждый результат findall можно итеративно обойти, распечатав тэг и его значение в виде
<pre>
        for i in movie:
            print(i.text, i.tag)
</pre>
* из полученного словаря сформируйте DataFrame с тремя колонками: `movie_id | tag | value` (movie_id соответствует номеру словаря среди всех словарей `content_list`)
* сохраните Dataframe c помощью функции `.to_csv()`

In [195]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree

# имя входного файла XML
input_file = './data/xml_content_description.xml'
# имя выходного файла CSV
output_file = 'csv_content_description.csv'
content_list = []

with open(input_file) as f:
    doc = ElementTree.parse(f)
    content_titles = doc.getroot()
    for movie in content_titles.findall('./Content/content_title'):
        cur_dict = dict()
        for i in movie:
            cur_dict.setdefault(i.tag, i.text)
        content_list.append(cur_dict)

# Почему-то варианты с append-ом у меня не работали.
# df = pd.DataFrame(columns = ['movie_id', 'tag', 'value'])
# for counter, cur_dict in enumerate(content_list):
#    d = {'movie_id': counter, 'tag': list(cur_dict.keys()), 'value': list(cur_dict.values())}
#     df_tmp = pd.DataFrame(data=d)
#     df.append(df_tmp)
#     df.append(d, ignore_index=True)
        
df = pd.concat([pd.DataFrame(data={'movie_id': counter, 
                              'tag': list(cur_dict.keys()), 
                              'value': list(cur_dict.values())}, 
                        columns=['movie_id', 'tag', 'value']) for counter, cur_dict in enumerate(content_list)],
          ignore_index=True)
df.to_csv(output_file)